<a href="https://colab.research.google.com/github/chltjs1921/AI-Predictor-of-LC-Retention-Time-for-Dansylated-Metabolites/blob/main/0616_RT_ANN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
##Dataset edit

import pandas as pd

from sklearn.feature_selection import VarianceThreshold, GenericUnivariateSelect, f_regression

#Enter input data
df = pd.read_csv("/content/drive/MyDrive/RT_predict/Urine_DnS/Dataset/0607/0607MORDRED2D.csv")

#Split data X and Y
dataset = df.values
rawX = dataset[:,0:-1]
Y = dataset[:,-1]

#Feature selection

selector = VarianceThreshold(threshold=0.001)
X1 = selector.fit_transform(rawX)
#print(X1.shape) 

X2 = GenericUnivariateSelect(f_regression, mode='percentile', param=0.5).fit_transform(X1, Y)
#print(X2.shape)  
#X2
pd.DataFrame(X2).to_csv("/content/drive/MyDrive/RT_predict/Urine_DnS/Dataset/0628MORDRED2D_edit.csv", index=False)

In [ ]:
##Make model and save

import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Activation, InputLayer
from tensorflow.python.keras.utils import np_utils
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.feature_selection import VarianceThreshold, GenericUnivariateSelect, f_regression
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from pickle import dump

#Generate seed
seed = 0
np.random.seed(seed)

#Enter the preprocessed data
df = pd.read_csv("/content/drive/MyDrive/RT_predict/Urine_DnS/Checkpoint/0616MORDRED2D_edit.csv")

#Split data X and Y
dataset = df.values
X = dataset[:,0:-1]
Y = dataset[:,-1]

#Split data train and test set

X_train, X_test, Y_train, Y_test = train_test_split(X, Y, test_size=0.2, random_state=seed)
X_train_df = pd.DataFrame(X_train)
X_train_df.to_csv("/content/drive/MyDrive/RT_predict/Urine_DnS/Checkpoint/0616MORDRED2D_edit_train.csv")
X_test_df = pd.DataFrame(X_test)
X_test_df.to_csv("/content/drive/MyDrive/RT_predict/Urine_DnS/Checkpoint/0616MORDRED2D_edit_test.csv")

#X Normalization

scaler = StandardScaler()

scaler.fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

#Create model

input_dim = 58

model = Sequential()
model.add(Dense(58, input_dim=input_dim, activation='relu'))
model.add(Dropout(rate=0.5))
#model.add(Dense(60, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse'])

model.fit(X_train_scaled, Y_train, epochs=200, batch_size=5)

#Compare(make it to dataaframe)

Y_pred = model.predict(X_test_scaled).flatten()
for i in range(len(X_test_scaled)):
    label = Y_test[i]
    pred = round(Y_pred[i],2)
    print("Real RT:{:.2f}, Predicted RT:{:.2f}".format(label, pred))

print(model.evaluate(X_test_scaled, Y_test))    

# test_compare = pd.DataFrame(columns=['Experimental_RT','Predicted_RT'])
# for i in range(len(X_test_scaled)):
#     Experimental_RT = Y_test[i]
#     Predicted_RT = round(Y_pred[i],2)
#     test_compare = test_compare.append(pd.DataFrame([[Experimental_RT,Predicted_RT]], columns=['Experimental_RT','Predicted_RT']), ignore_index=True)
# test_compare.to_csv("/content/drive/MyDrive/RT_predict/Urine_DnS/Checkpoint/0628/0628_RT_testprediction9.csv", index=None)   

# Y_train_pred = model.predict(X_train_scaled).flatten()
# train_compare = pd.DataFrame(columns=['Experimental_RT','Predicted_RT'])
# for i in range(len(X_train_scaled)):
#     Experimental_RT = Y_train[i]
#     Predicted_RT = round(Y_train_pred[i],2)
#     train_compare = train_compare.append(pd.DataFrame([[Experimental_RT,Predicted_RT]], columns=['Experimental_RT','Predicted_RT']), ignore_index=True)
# train_compare.to_csv("/content/drive/MyDrive/RT_predict/Urine_DnS/Checkpoint/0628/0628_RT_trainprediction9.csv", index=None)

# #Save model

# dump(scaler, open("/content/drive/MyDrive/RT_predict/Urine_DnS/Checkpoint/0628/RT_predict_0628_9.pkl", "wb"))
# model.save("/content/drive/MyDrive/RT_predict/Urine_DnS/Checkpoint/0628/RT_predict_0628_9.h5")

Epoch 1/200
51/51 [==============================] - 1s 2ms/step - loss: 180.6615 - mae: 11.9513 - mse: 180.6615
Epoch 2/200
51/51 [==============================] - 0s 2ms/step - loss: 92.9618 - mae: 8.2421 - mse: 92.9618
Epoch 3/200
51/51 [==============================] - 0s 2ms/step - loss: 45.0603 - mae: 5.4845 - mse: 45.0603
Epoch 4/200
51/51 [==============================] - 0s 2ms/step - loss: 35.3657 - mae: 4.7845 - mse: 35.3657
Epoch 5/200
51/51 [==============================] - 0s 2ms/step - loss: 30.2897 - mae: 4.3698 - mse: 30.2897
Epoch 6/200
51/51 [==============================] - 0s 2ms/step - loss: 31.3649 - mae: 4.2803 - mse: 31.3649
Epoch 7/200
51/51 [==============================] - 0s 1ms/step - loss: 24.3517 - mae: 3.8930 - mse: 24.3517
Epoch 8/200
51/51 [==============================] - 0s 2ms/step - loss: 21.4386 - mae: 3.5739 - mse: 21.4386
Epoch 9/200
51/51 [==============================] - 0s 2ms/step - loss: 23.3258 - mae: 3.7430 - mse: 23.3258
Epoch 1

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.python.keras.models import Sequential, load_model
from tensorflow.python.keras.layers import Dense, Dropout, Activation, InputLayer
from tensorflow.python.keras.utils import np_utils
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.feature_selection import VarianceThreshold, GenericUnivariateSelect, f_regression
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from pickle import dump

#Create seed
seed = 0
np.random.seed(seed)

#Enter preprocessed data
df = pd.read_csv("/content/drive/MyDrive/RT_predict/Urine_DnS/Checkpoint/0616MORDRED2D_edit.csv")

#Date split X and Y
dataset = df.values
X = dataset[:,0:-1]
Y = dataset[:,-1]

#Load model and scaler
#scaler = load(open('RT_predict_0628_8.pkl', 'rb'))

#model = load_model('RT_predict_0628_8.h5')

#print(model.summary())

In [ ]:
##Create model and save

import pandas as pd
import numpy as np
import tensorflow as tf

from tensorflow.python.keras.models import Sequential
from tensorflow.python.keras.layers import Dense, Dropout, Activation, InputLayer
from tensorflow.python.keras.utils import np_utils
from sklearn.model_selection import train_test_split, cross_val_score, cross_validate
from sklearn.feature_selection import VarianceThreshold, GenericUnivariateSelect, f_regression
from sklearn.preprocessing import MinMaxScaler, StandardScaler
from sklearn.model_selection import StratifiedShuffleSplit, KFold
from pickle import dump

#Generate seed
seed = 0
np.random.seed(seed)

#Enter preproceesed data
df = pd.read_csv("/content/drive/MyDrive/RT_predict/Urine_DnS/Checkpoint/0616MORDRED2D_edit.csv")

#Split data X and Y
dataset = df.values
X = dataset[:,0:-1]
Y = dataset[:,-1]

In [ ]:
input_dim = 58

model = Sequential()
model.add(Dense(58, input_dim=input_dim, activation='relu'))
model.add(Dropout(rate=0.5))
#model.add(Dense(60, activation='relu'))
model.add(Dense(10, activation='relu'))
model.add(Dense(1))
model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse'])
model.summary()

#model.fit(X_train_scaled, Y_train, epochs=200, batch_size=5)

Model: "sequential_3"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_9 (Dense)              (None, 58)                3422      
_________________________________________________________________
dropout_3 (Dropout)          (None, 58)                0         
_________________________________________________________________
dense_10 (Dense)             (None, 10)                590       
_________________________________________________________________
dense_11 (Dense)             (None, 1)                 11        
Total params: 4,023
Trainable params: 4,023
Non-trainable params: 0
_________________________________________________________________


In [ ]:
kf = KFold(n_splits=5, shuffle=True, random_state=seed)
for train_index, test_index in kf.split(X, Y):
    print("TRAIN:", train_index, "TEST:", test_index)
    X_train, X_test = X[train_index], X[test_index]
    Y_train, Y_test = Y[train_index], Y[test_index]

TRAIN: [  0   1   2   3   4   6   9  10  11  13  14  16  18  19  20  23  24  25
  27  28  29  30  31  32  34  35  36  37  38  39  40  41  42  43  44  45
  46  47  48  49  50  51  53  54  57  58  61  62  64  67  68  69  70  71
  72  73  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  93  94  95  96  98  99 100 102 103 104 105 106 107 109 110 111 112
 113 114 115 117 118 119 121 122 123 125 127 128 130 131 134 136 138 139
 140 141 143 146 147 148 149 151 153 154 155 157 159 160 161 162 163 165
 166 167 168 169 170 172 173 174 176 177 178 179 180 181 182 183 184 185
 186 187 188 189 190 191 192 193 194 195 196 197 198 199 200 201 202 203
 204 205 206 207 208 209 210 211 212 213 214 215 216 218 219 220 222 223
 224 225 226 227 229 231 232 233 234 236 237 238 239 242 243 244 245 246
 248 251 254 256 257 259 260 263 264 265 266 267 268 269 270 271 272 273
 274 276 277 278 279 280 282 283 284 285 286 287 288 289 290 291 292 293
 294 295 296 297 298 299 300 301 302 303 304

In [ ]:
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)
dump(scaler, open("/content/drive/MyDrive/RT_predict/Urine_DnS/Checkpoint/230314/0314scaler.pkl", "wb"))

In [ ]:
for train_idx, test_idx in kf.split(X, Y):
    # Split the data into training and testing sets
    X_train, X_test = X[train_idx], X[test_idx]
    Y_train, Y_test = Y[train_idx], Y[test_idx]

    # Standardize the features using the scaler fit on the training data
    X_train = scaler.transform(X_train)
    X_test = scaler.transform(X_test)

    # Compile the model and train it on the training data
    model.compile(loss='mean_squared_error', optimizer='adam', metrics=['mae', 'mse'])
    model.fit(X_train, Y_train, batch_size=5, epochs=200, validation_data=(X_test, Y_test))

    # Evaluate the model on the testing data
    score = model.evaluate(X_test, Y_test)
    print(f"Accuracy on test set: {score}")

Epoch 1/200
51/51 [==============================] - 1s 6ms/step - loss: 178.0870 - mae: 11.6640 - mse: 178.0870 - val_loss: 123.1126 - val_mae: 9.4500 - val_mse: 123.1126
Epoch 2/200
51/51 [==============================] - 0s 3ms/step - loss: 92.6171 - mae: 8.2132 - mse: 92.6171 - val_loss: 45.4237 - val_mae: 5.5437 - val_mse: 45.4237
Epoch 3/200
51/51 [==============================] - 0s 3ms/step - loss: 56.6340 - mae: 6.2345 - mse: 56.6340 - val_loss: 25.5696 - val_mae: 4.2230 - val_mse: 25.5696
Epoch 4/200
51/51 [==============================] - 0s 2ms/step - loss: 40.5134 - mae: 5.0360 - mse: 40.5134 - val_loss: 17.5810 - val_mae: 3.5102 - val_mse: 17.5810
Epoch 5/200
51/51 [==============================] - 0s 3ms/step - loss: 34.0538 - mae: 4.6918 - mse: 34.0538 - val_loss: 14.2766 - val_mae: 3.1107 - val_mse: 14.2766
Epoch 6/200
51/51 [==============================] - 0s 3ms/step - loss: 32.8320 - mae: 4.5246 - mse: 32.8320 - val_loss: 12.0240 - val_mae: 2.8229 - val_mse: 1

In [ ]:
#Compare(make to dataaframe)
compare = pd.DataFrame(columns=['Experimental_RT','Predicted_RT'])
Y_pred = model.predict(X_scaled).flatten()
for i in range(len(X_scaled)):
    label = Y[i]
    pred = round(Y_pred[i],2)
    compare = compare.append(pd.DataFrame([[label,pred]], columns=['Experimental_RT','Predicted_RT']), ignore_index=True)
    print("Real RT:{:.2f}, Predicted RT:{:.2f}".format(label, pred))
compare.to_csv("/content/drive/MyDrive/RT_predict/Urine_DnS/Checkpoint/230314/0314_RT_totalprediction.csv", index=None)

print(model.evaluate(X_scaled, Y))    

test_compare = pd.DataFrame(columns=['Experimental_RT','Predicted_RT'])
Y_pred_test = model.predict(X_test).flatten()
for i in range(len(X_test)):
    Experimental_RT = Y_test[i]
    Predicted_RT = round(Y_pred_test[i],2)
    test_compare = test_compare.append(pd.DataFrame([[Experimental_RT,Predicted_RT]], columns=['Experimental_RT','Predicted_RT']), ignore_index=True)
test_compare.to_csv("/content/drive/MyDrive/RT_predict/Urine_DnS/Checkpoint/230314/0314_RT_testprediction.csv", index=None)   

Y_train_pred = model.predict(X_train).flatten()
train_compare = pd.DataFrame(columns=['Experimental_RT','Predicted_RT'])
for i in range(len(X_train)):
    Experimental_RT = Y_train[i]
    Predicted_RT = round(Y_train_pred[i],2)
    train_compare = train_compare.append(pd.DataFrame([[Experimental_RT,Predicted_RT]], columns=['Experimental_RT','Predicted_RT']), ignore_index=True)
train_compare.to_csv("/content/drive/MyDrive/RT_predict/Urine_DnS/Checkpoint/230314/0314_RT_trainprediction.csv", index=None)

#Save model

model.save("/content/drive/MyDrive/RT_predict/Urine_DnS/Checkpoint/230314/RT_predict_230314.h5")

<ipython-input-35-0d4b37fee224>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  compare = compare.append(pd.DataFrame([[label,pred]], columns=['Experimental_RT','Predicted_RT']), ignore_index=True)
<ipython-input-35-0d4b37fee224>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  compare = compare.append(pd.DataFrame([[label,pred]], columns=['Experimental_RT','Predicted_RT']), ignore_index=True)
<ipython-input-35-0d4b37fee224>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  compare = compare.append(pd.DataFrame([[label,pred]], columns=['Experimental_RT','Predicted_RT']), ignore_index=True)
<ipython-input-35-0d4b37fee224>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future vers

실제 RT:2.17, 예상 RT:3.48
실제 RT:2.63, 예상 RT:3.50
실제 RT:20.49, 예상 RT:19.66
실제 RT:16.91, 예상 RT:17.99
실제 RT:23.88, 예상 RT:19.80
실제 RT:25.49, 예상 RT:23.33
실제 RT:1.75, 예상 RT:3.84
실제 RT:3.94, 예상 RT:4.32
실제 RT:5.82, 예상 RT:6.80
실제 RT:7.24, 예상 RT:8.21
실제 RT:3.02, 예상 RT:3.70
실제 RT:13.23, 예상 RT:13.74
실제 RT:8.49, 예상 RT:8.90
실제 RT:15.07, 예상 RT:12.61
실제 RT:5.87, 예상 RT:4.21
실제 RT:7.38, 예상 RT:7.62
실제 RT:1.88, 예상 RT:3.59
실제 RT:2.87, 예상 RT:3.85
실제 RT:13.34, 예상 RT:13.79
실제 RT:13.69, 예상 RT:13.83
실제 RT:8.72, 예상 RT:8.12
실제 RT:7.79, 예상 RT:7.89
실제 RT:13.57, 예상 RT:15.54
실제 RT:16.51, 예상 RT:17.05
실제 RT:6.59, 예상 RT:7.94
실제 RT:2.74, 예상 RT:3.84
실제 RT:24.84, 예상 RT:21.97
실제 RT:2.22, 예상 RT:4.44
실제 RT:5.05, 예상 RT:6.24
실제 RT:9.46, 예상 RT:9.59
실제 RT:6.00, 예상 RT:5.92
실제 RT:17.11, 예상 RT:18.70
실제 RT:24.69, 예상 RT:22.30
실제 RT:20.36, 예상 RT:22.76
실제 RT:2.12, 예상 RT:3.77
실제 RT:8.73, 예상 RT:9.23
실제 RT:9.65, 예상 RT:9.59
실제 RT:22.65, 예상 RT:20.88
실제 RT:12.74, 예상 RT:13.33
실제 RT:7.57, 예상 RT:8.85
실제 RT:10.18, 예상 RT:11.37
실제 RT:9.82, 예상 RT:8.63


<ipython-input-35-0d4b37fee224>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  compare = compare.append(pd.DataFrame([[label,pred]], columns=['Experimental_RT','Predicted_RT']), ignore_index=True)
<ipython-input-35-0d4b37fee224>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  compare = compare.append(pd.DataFrame([[label,pred]], columns=['Experimental_RT','Predicted_RT']), ignore_index=True)
<ipython-input-35-0d4b37fee224>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  compare = compare.append(pd.DataFrame([[label,pred]], columns=['Experimental_RT','Predicted_RT']), ignore_index=True)
<ipython-input-35-0d4b37fee224>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future vers


실제 RT:3.74, 예상 RT:3.70
실제 RT:4.58, 예상 RT:3.76
실제 RT:11.44, 예상 RT:12.12
실제 RT:10.52, 예상 RT:11.88
실제 RT:18.47, 예상 RT:18.91
실제 RT:17.49, 예상 RT:18.65
실제 RT:26.70, 예상 RT:24.46
실제 RT:2.47, 예상 RT:3.56
실제 RT:2.94, 예상 RT:4.14
실제 RT:5.98, 예상 RT:4.68
실제 RT:19.20, 예상 RT:18.73
실제 RT:5.57, 예상 RT:4.16
실제 RT:8.62, 예상 RT:6.19
실제 RT:10.77, 예상 RT:11.51
실제 RT:13.77, 예상 RT:14.43
실제 RT:8.73, 예상 RT:8.84
실제 RT:16.62, 예상 RT:13.87
실제 RT:7.59, 예상 RT:7.47
실제 RT:16.30, 예상 RT:16.80
실제 RT:25.04, 예상 RT:23.08
실제 RT:6.97, 예상 RT:7.61
실제 RT:4.66, 예상 RT:3.60
실제 RT:23.45, 예상 RT:22.68
실제 RT:14.32, 예상 RT:15.80
실제 RT:1.60, 예상 RT:3.31
실제 RT:10.54, 예상 RT:9.83
실제 RT:23.90, 예상 RT:22.18
실제 RT:1.49, 예상 RT:3.08
실제 RT:12.30, 예상 RT:13.67
실제 RT:12.96, 예상 RT:13.76
실제 RT:11.52, 예상 RT:12.65
실제 RT:1.15, 예상 RT:3.84
실제 RT:22.54, 예상 RT:22.12
실제 RT:1.48, 예상 RT:3.13
실제 RT:1.28, 예상 RT:3.41
실제 RT:21.27, 예상 RT:19.72
실제 RT:19.47, 예상 RT:19.07
실제 RT:4.52, 예상 RT:3.70
실제 RT:15.28, 예상 RT:13.96
실제 RT:18.14, 예상 RT:15.20
실제 RT:3.84, 예상 RT:4.28
실제 RT:22.52

<ipython-input-35-0d4b37fee224>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  compare = compare.append(pd.DataFrame([[label,pred]], columns=['Experimental_RT','Predicted_RT']), ignore_index=True)
<ipython-input-35-0d4b37fee224>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  compare = compare.append(pd.DataFrame([[label,pred]], columns=['Experimental_RT','Predicted_RT']), ignore_index=True)
<ipython-input-35-0d4b37fee224>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  compare = compare.append(pd.DataFrame([[label,pred]], columns=['Experimental_RT','Predicted_RT']), ignore_index=True)
<ipython-input-35-0d4b37fee224>:7: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future vers

[1.911579966545105, 1.0260764360427856, 1.911579966545105]


<ipython-input-35-0d4b37fee224>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_compare = test_compare.append(pd.DataFrame([[Experimental_RT,Predicted_RT]], columns=['Experimental_RT','Predicted_RT']), ignore_index=True)
<ipython-input-35-0d4b37fee224>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_compare = test_compare.append(pd.DataFrame([[Experimental_RT,Predicted_RT]], columns=['Experimental_RT','Predicted_RT']), ignore_index=True)
<ipython-input-35-0d4b37fee224>:18: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  test_compare = test_compare.append(pd.DataFrame([[Experimental_RT,Predicted_RT]], columns=['Experimental_RT','Predicted_RT']), ignore_index=True)
<ipython-input-35-0d4b37fee224>:18: FutureWarning

In [ ]:
Y_train_pred = model.predict(X_train).flatten()
train_compare = pd.DataFrame(columns=['Experimental_RT','Predicted_RT'])
for i in range(len(X_train)):
    Experimental_RT = Y_train[i]
    Predicted_RT = round(Y_train_pred[i],2)
    train_compare = train_compare.append(pd.DataFrame([[Experimental_RT,Predicted_RT]], columns=['Experimental_RT','Predicted_RT']), ignore_index=True)
train_compare.to_csv("/content/drive/MyDrive/RT_predict/Urine_DnS/Checkpoint/230314/0314_RT_trainprediction.csv", index=None)